In [1]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from resnet3d import resnet, mean
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid
from activation_maximization import generate_preferred


In [38]:
net = resnet.resnet50(num_classes=400, shortcut_type='B', sample_size=112, sample_duration=16)
net = torch.nn.DataParallel(net, device_ids=None)
param_file = os.path.join('../model','resnet3d', 'resnet-50-kinetics.pth')
param = torch.load(param_file, map_location='cpu')
net.load_state_dict(param['state_dict'])
net.eval()


../model/resnet3d/resnet.py:159: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


DataParallel(
  (module): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (ide

In [28]:
img_mean = np.float32([104, 117, 123])
img_std = np.float32([1,1,1])

# image
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

img_mean = mean.get_mean(1)
img_std = mean.get_std(1)

In [22]:
img_std

[38.7568578, 37.88248729, 40.02898126]

In [39]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_resnet3D_TV_norm_all'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [40]:
def hook(module, input, output):
    outputs.append(output.clone())

In [41]:
# initial image for the optimization
fr, h, w = 16, 112,112

initial_input = np.random.randint(0, 256, (fr, h,w,3))

In [42]:
net

DataParallel(
  (module): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (ide

In [45]:
target_layer = "net.layer1[2].conv3"
target_layer = "net.module.conv1"
target_layer = "net.module.layer4[0].bn3"
target_layer = "net.module.layer1[2].conv3"

In [46]:
#targetlayer
outputs = []
exec_str_list = [target_layer +".register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = "+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)
ee = net(torch.Tensor(initial_input.transpose(3,0, 1, 2)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [47]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (32, 112,112,3),
    #'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }




In [48]:
for channel in channel_list[:2]:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    t_index = int(feat_size[2]/2)
    y_index = int(feat_size[3]/2) # the unit in the center of feature map
    x_index = int(feat_size[4]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,t_index,y_index,x_index] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.

    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=0

iter=1; mean(abs(feat))=0.000291529;
iter=2; mean(abs(feat))=0.00715177;
iter=3; mean(abs(feat))=0.00878274;
iter=4; mean(abs(feat))=0.00414184;
iter=5; mean(abs(feat))=0.011772;
iter=6; mean(abs(feat))=0.00814653;
iter=7; mean(abs(feat))=0.0129455;
iter=8; mean(abs(feat))=0.0205809;
iter=9; mean(abs(feat))=0.0183472;
iter=10; mean(abs(feat))=0.018599;
iter=11; mean(abs(feat))=0.0142927;
iter=12; mean(abs(feat))=0.0252574;
iter=13; mean(abs(feat))=0.019405;
iter=14; mean(abs(feat))=0.0188842;
iter=15; mean(abs(feat))=0.011177;
iter=16; mean(abs(feat))=0.0181988;
iter=17; mean(abs(feat))=0.0219473;
iter=18; mean(abs(feat))=0.0245365;
iter=19; mean(abs(feat))=0.0239923;
iter=20; mean(abs(feat))=0.0204981;
iter=21; mean(abs(feat))=0.0262748;
iter=22; mean(abs(feat))=0.0314092;
iter=23; mean(abs(feat))=0.0245367;
iter=24; mean(abs(feat))=0.0339685;
iter=25; mean(abs(feat))=0.0372935;
iter=26; mean(abs(feat))=0.0267696;
iter=27; mean(abs(feat))=0.0277009;
iter=28; mean(abs(feat)

NameError: name 'layer' is not defined

In [33]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 32,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': True,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (32, 112,112,3),
    #'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }

In [37]:
for channel in channel_list[-2:]:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
     # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.



    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=320

iter=1; mean(abs(feat))=0.166597;
iter=2; mean(abs(feat))=0.560629;
iter=3; mean(abs(feat))=1.54964;
iter=4; mean(abs(feat))=1.48757;
iter=5; mean(abs(feat))=1.38963;
iter=6; mean(abs(feat))=1.661;
iter=7; mean(abs(feat))=2.05572;
iter=8; mean(abs(feat))=1.75627;
iter=9; mean(abs(feat))=2.10815;
iter=10; mean(abs(feat))=2.08928;
iter=11; mean(abs(feat))=1.85591;
iter=12; mean(abs(feat))=1.67454;
iter=13; mean(abs(feat))=1.85124;
iter=14; mean(abs(feat))=1.69539;
iter=15; mean(abs(feat))=2.1298;
iter=16; mean(abs(feat))=1.75607;
iter=17; mean(abs(feat))=2.45579;
iter=18; mean(abs(feat))=1.80524;
iter=19; mean(abs(feat))=1.79769;
iter=20; mean(abs(feat))=2.20533;
iter=21; mean(abs(feat))=1.87931;
iter=22; mean(abs(feat))=2.23086;
iter=23; mean(abs(feat))=2.21061;
iter=24; mean(abs(feat))=1.81229;
iter=25; mean(abs(feat))=1.98451;
iter=26; mean(abs(feat))=2.01831;
iter=27; mean(abs(feat))=2.01521;
iter=28; mean(abs(feat))=1.78775;
iter=29; mean(abs(feat))=2.08087;
iter=30; m

iter=40; mean(abs(feat))=7.01867;
iter=41; mean(abs(feat))=7.80424;
iter=42; mean(abs(feat))=9.11227;
iter=43; mean(abs(feat))=7.90331;
iter=44; mean(abs(feat))=7.36755;
iter=45; mean(abs(feat))=6.56438;
iter=46; mean(abs(feat))=5.79533;
iter=47; mean(abs(feat))=8.10529;
iter=48; mean(abs(feat))=5.69435;
iter=49; mean(abs(feat))=5.52198;
iter=50; mean(abs(feat))=7.35886;
iter=51; mean(abs(feat))=6.58387;
iter=52; mean(abs(feat))=5.51354;
iter=53; mean(abs(feat))=7.40873;
iter=54; mean(abs(feat))=5.37663;
iter=55; mean(abs(feat))=6.21987;
iter=56; mean(abs(feat))=6.91364;
iter=57; mean(abs(feat))=6.28513;
iter=58; mean(abs(feat))=6.20385;
iter=59; mean(abs(feat))=6.25432;
iter=60; mean(abs(feat))=5.40337;
iter=61; mean(abs(feat))=6.85843;
iter=62; mean(abs(feat))=6.50164;
iter=63; mean(abs(feat))=6.58848;
iter=64; mean(abs(feat))=7.77185;
iter=65; mean(abs(feat))=7.1517;
iter=66; mean(abs(feat))=6.86051;
iter=67; mean(abs(feat))=7.3051;
iter=68; mean(abs(feat))=6.27543;
iter=69; mean(ab

In [35]:
target_layer

'net.module.fc'